In [30]:
import sys
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
from helper_functions import data_reader

In [31]:
# load all relevant data
df_CF = data_reader.getCFs()
df_ecoregions = data_reader.getEcoregions()
df_miningPolygons = data_reader.getMiningPolygons()
df_miningFacilities = data_reader.getMiningFacilities()
df_miningMinerals = data_reader.getMiningMinerals()
df_miningCommodities = data_reader.getMiningCommodities()
df_miningSourceIDs = data_reader.getSourceIDs()
df_miningMaterialIDs = data_reader.getMaterialIDs()

Merge the CF and the ecoregions

In [ ]:
# Unify datatype of ecoregion ids to ensure correct join
df_CF['eco_id'] = df_CF['eco_id'].astype(int)
df_ecoregions['ECO_ID'] = df_ecoregions['ECO_ID'].astype(int)

# Group CFs by ecoregion into list of dicts
#TODO: Maybe I should aggregate the CF's already here to get one CF per ecoregion
df_CF_grouped = (
    df_CF.groupby("eco_id")
    .apply(lambda g: g.to_dict(orient="records"), include_groups=False)  # list of row-dicts
    .reset_index(name="cf_data")  # put it into a column
)

# Merge CFs with ecoregions to get spatial representation of CFs
df_CF_ecoregions = df_ecoregions.merge(df_CF_grouped, left_on='ECO_ID', right_on='eco_id')

# Check which ecoregions have a match in df_CF
matched_ids = df_CF['eco_id'].unique()
all_ids = df_ecoregions['ECO_ID'].unique()
missing_ids = np.setdiff1d(all_ids, matched_ids)
unmatched_names = df_ecoregions[df_ecoregions['ECO_ID'].isin(missing_ids)]['ECO_NAME'].unique()
print(f"CF has {len(matched_ids)} ecoregions and in total there would be {len(all_ids)}.\n"
    f"The unmatched ecoregions are {missing_ids}, {unmatched_names} which are now absent in the further analysis.")

# Check if crs is EPSG4326, change it otherwise
if df_CF_ecoregions.crs != "EPSG:4326":
    df_CF_ecoregions = df_CF_ecoregions.to_crs("EPSG:4326")
else:
    print("CRS is already EPSG:4326")

CF has 825 ecoregions and in total there would be 827.
The unmatched ecoregions are [-9999 -9998], ['Rock and Ice' 'Lake'] which are now absent in the further analysis.
CRS is already EPSG:4326


# Test

In [ ]:
# Count non-null values for each column in df_miningFacilities
non_null_counts = df_miningFacilities.notnull().sum().to_frame(name='non_null_count')
non_null_counts['percent'] = round((non_null_counts['non_null_count'] / len(df_miningFacilities)) * 100, 1)
print(non_null_counts)
print(df_miningFacilities['facility_type'].unique())
#TODO: Are all facility types relevant?

print(df_miningFacilities['commodities_products'].unique())
#TODO: Are all commodities relevant? -> delete coal, oil?

#TODO: The numbers don't match with the 1171 mines mentioned in the paper?

                       non_null_count  percent
facility_id                      2413    100.0
facility_name                    2413    100.0
facility_other_names              397     16.5
sub_site_name                     978     40.5
sub_site_other_names              102      4.2
facility_type                    2413    100.0
primary_commodity                2413    100.0
commodities_products             2333     96.7
facility_equipment               1345     55.7
production_start                  598     24.8
production_end                    117      4.8
activity_status                    69      2.9
activity_status_year               64      2.7
surface_area_sq_km                 15      0.6
concession_area_sq_km             129      5.3
country                          2413    100.0
GID_0                            2413    100.0
GID_1                            1435     59.5
GID_2                            1424     59.0
GID_3                             790     32.7
GID_4        

In [ ]:
# Count non-null values for each column in df_miningMinerals
non_null_counts = df_miningMinerals.notnull().sum().to_frame(name='non_null_count')
non_null_counts['percent'] = round((non_null_counts['non_null_count'] / len(df_miningMinerals)) * 100, 1)
print(non_null_counts)
print(df_miningMinerals['facility_id'].nunique(), "unique facility ids in minerals")
print(df_miningMinerals['material'].unique(), "unique materials in minerals")
#TODO: Are all materials relevant? -> delete coal, oil?
print(df_miningMinerals['type'].unique(), "unique types in minerals")
#TODO: What is this exactly?


                    non_null_count  percent
id                           12292    100.0
facility_id                  12292    100.0
year                         12292    100.0
type                         12292    100.0
material                     12292    100.0
value_tonnes                 12292    100.0
overall_grade_ppm              408      3.3
amount_sold_tonnes             542      4.4
mine_processing                483      3.9
reporting_period              1960     15.9
source_id                    12292    100.0
comment                        655      5.3
570 unique facility ids in minerals
['O.bulk' 'O.Fe' 'O.Cu' 'O.Au' 'Con.Cu' 'O.oxi' 'O.sul' 'O.Al' 'Oth.sif'
 'O.P' 'Con.Fe' 'O.FeTiO3' 'O.TiO2' 'O.Zr' 'O.Mn' 'O.Ni' 'Con.MnO'
 'Con.Pb' 'Con.Zn' 'M.dol' 'M.lim' 'O.Ag' 'Con.Mo' 'O.Nb' 'O.FeS'
 'Con.bulk' 'O.Sn' 'Con.MnCO3' 'Con.Cr' 'O.Cr' 'O.dig' 'O.Zn' 'O.leu'
 'Con.Ni' 'O.nep' 'O.pgm' 'M.lat' 'M.kao' 'Con.Li' 'Con.Ta' 'O.Li' 'O.K2O'
 'Oth.cuce' 'O.flu'] unique materials in 

In [85]:
# Count non-null values for each column in df_miningCommodities
non_null_counts = df_miningCommodities.notnull().sum().to_frame(name='non_null_count')
non_null_counts['percent'] = round((non_null_counts['non_null_count'] / len(df_miningCommodities)) * 100, 1)
print(non_null_counts)
print(df_miningCommodities['facility_id'].nunique(), "unique facility ids in commodities")
print(df_miningCommodities['id_minerals'].nunique(), "unique mineral ids in commodities")
print(df_miningCommodities['material'].nunique(), "unique materials in commodities")

                      non_null_count  percent
id                              9426    100.0
facility_id                     9426    100.0
year                            9426    100.0
material                        8895     94.4
commodity                       9426    100.0
value_tonnes                    9399     99.7
grade_ppm                       7132     75.7
recovery_rate                   5497     58.3
yield_ppm                        127      1.3
amount_sold_tonnes              1423     15.1
metal_payable_tonnes             429      4.6
mine_processing                  473      5.0
reporting_period                1233     13.1
source_id                       9426    100.0
comment                          386      4.1
id_minerals                     7053     74.8
507 unique facility ids in commodities
4046 unique mineral ids in commodities
29 unique materials in commodities


In [52]:
# spatially merge the dataframes miningPolygons and miningFacilities
df_miningPolygons = df_miningPolygons.to_crs("EPSG:4326")
df_miningFacilities = df_miningFacilities.to_crs("EPSG:4326")
df_miningFacilities_polygons = gpd.sjoin(df_miningFacilities, df_miningPolygons, how="inner", predicate='within')
print(len(df_miningPolygons), "mining polygons")
print(len(df_miningFacilities), "mining facilities")
print(f"Number of mining facilities after spatial join: {len(df_miningFacilities_polygons)}")

44929 mining polygons
2413 mining facilities
Number of mining facilities after spatial join: 1113
